In [1]:
import os.path
import os
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from google.oauth2.credentials  import Credentials
import pandas as pd
import numpy as np
from datetime import datetime
import re
import sys
import glob
import subprocess

In [2]:
# In[44]:
# ID and range of a seq processing sheet
                       # 1S_7ql04XgKSu3VTRRiOyrCmzP8GhOEhAzxdByhpgj1A
SAMPLE_SPREADSHEET_ID = '1S_7ql04XgKSu3VTRRiOyrCmzP8GhOEhAzxdByhpgj1A'
#SAMPLE_RANGE_NAME = 'A1:M10000'
now = datetime.now()

# In[47]:
# If modifying these scopes, delete the file token.json.
SCOPES = ['https://www.googleapis.com/auth/spreadsheets.readonly']

In [ ]:
### We need functions too:
# 1. Read google sheet.
# 2. Create pepatac datasheet from input rows 
# 3. Extract project name from google sheet given input rows 
# 4. Extract Input path from google sheet given input rows 
# 5. Parse datasheet to create R1, R2 paths from input path
# 6. Edit yaml with relevant info

### Note: See if we can add extra if organism then conditions to yaml

### Extras 
# Format bash script to launch yaml

In [ ]:
#### Edit yaml to incluse 
# 1. project name 
# 2. sample_table_path
# 3. output_path
# 4. R1 path
# 5. R2 path
# 6. organism
# 7. genome index
# 8. organism chrom sizes 

In [3]:
def read_atac_sheet():
    # ATAC sheet ID
   # SAMPLE_SPREADSHEET_ID = '1S_7ql04XgKSu3VTRRiOyrCmzP8GhOEhAzxdByhpgj1A'
   # SAMPLE_RANGE_NAME = 'A1:M10000'
   # now = datetime.now()
   # SCOPES = ['https://www.googleapis.com/auth/spreadsheets.readonly']
    
    creds = None
    # The file token.json stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists('/projects/ps-epigen/users/rlan/ATACSeq/bot/token.json'):
        creds = Credentials.from_authorized_user_file('/projects/ps-epigen/users/rlan/ATACSeq/bot/token.json', SCOPES)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                '/projects/ps-epigen/10x_output/Sequencing-Processing-Bot/python/credentials.json', SCOPES)
            creds = flow.run_local_server(port=5734)
        # Save the credentials for the next run
        with open('/projects/ps-epigen/users/rlan/ATACSeq/bot/token.json', 'w') as token:
            token.write(creds.to_json())

    service = build('sheets', 'v4', credentials=creds)

    # Call the Sheets API
    sheet = service.spreadsheets()
    result = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID,
                                range= 'PEPATAC!A1:M10000').execute()
    values = result.get('values', [])

    if not values:
        print('No data found.')
    else:
        DF = pd.DataFrame(values[1:], columns=values[0])
        
    return DF

In [13]:
DF = read_atac_sheet()

In [8]:
#DF

In [4]:
########### Use sample name to grep input dir to aquire the true seq name for data sheet #########

def create_pep_sheet(pandas_gs, start_row, stop_row):
    # rows = 20:24
    project = pandas_gs[start_row:stop_row]
    
    # 1. Create project dirs using the Base path + Data productions lead + project names
    base_path = "/projects/ps-epigen/users/rlan/ATACSeq/PEPATAC/FFS/"
    dlead = DF[start_row:start_row + 1].squeeze()[0]
    pname = DF[start_row:start_row + 1].squeeze()[1]
    proj_path = base_path + dlead + "/" + pname + "/"
    os.makedirs(proj_path, exist_ok = True)
    
    d = {'sample_name': project['SEQ_ID'] , 'protocol': 'ATAC' , 'organism': project['Organism'], 
         'read1': 'R1' , 'read2': 'R2' , 'read_type': 'paired'}
    pep_sheet = pd.DataFrame(d)
    
    #Use sample name to grep input dir to aquire the true seq name for data sheet
    ipath = DF[start_row:start_row + 1].squeeze()[6]
 #   print(ipath)
    #"/projects/ps-epigen/users/rlan/ATACSeq/demultiplex/221005_NB501692_0223_AH23H5AFX5/out/AH23H5AFX5/"
    ifiles = os.listdir(ipath)
    fbnames = list()

    for s in pep_sheet['sample_name']:
        fname = list(filter(lambda x: re.search(s + r".*R1", x), ifiles))
        fbase_name = re.search(r'^.*_R', fname[0]).group(0)[:-2]
        fbnames.append(fbase_name)

    pep_sheet['sample_name'] = fbnames
    pep_sheet.to_csv(path_or_buf= f"{proj_path}datasheet.csv", header=True, index = False)
    
    return pep_sheet

# if sample not present in input dir 
# throw error 

In [26]:
#pep_sheet = create_pep_sheet(DF, start_row=30, stop_row=42)
#pep_sheet = create_pep_sheet(DF, start_row=65, stop_row=80)
pep_sheet = create_pep_sheet(DF, start_row=168, stop_row=186)
pep_sheet

/projects/ps-epigen/users/rlan/ATACSeq/demultiplex/230105_NB501692_0239_AH5VCLAFX5/out/AH5VCLAFX5/


,sample_name,protocol,organism,read1,read2,read_type
168,QY_1861_S11,ATAC,human,R1,R2,paired
169,QY_1862_S12,ATAC,human,R1,R2,paired
170,QY_1863_S13,ATAC,human,R1,R2,paired
171,QY_1864_S14,ATAC,human,R1,R2,paired
172,QY_1865_S15,ATAC,human,R1,R2,paired
173,QY_1866_S16,ATAC,human,R1,R2,paired
174,QY_1867_S17,ATAC,human,R1,R2,paired
175,QY_1868_S18,ATAC,human,R1,R2,paired
176,QY_1869_S19,ATAC,human,R1,R2,paired
177,QY_1870_S20,ATAC,human,R1,R2,paired


In [5]:
def write_pepatac_shell(pep_DF, start_row):
    # extract input dir
    i_path = pep_DF[start_row:start_row + 1].squeeze()[6]
    
    base_path = "/projects/ps-epigen/users/rlan/ATACSeq/PEPATAC/FFS/"
    dlead = DF[start_row:start_row + 1].squeeze()[0]
    pname = DF[start_row:start_row + 1].squeeze()[1]
    proj_path = base_path + dlead + "/" + pname + "/"
    
    # Path to yaml = project_path + project.yaml
    ypath = proj_path + "project.yaml"
    
    # Replace strings in file with extracted idir & pdir
    with open("/projects/ps-epigen/users/rlan/ATACSeq/bot/pepatac.temp.sh", 'rt') as fh:
        data = fh.read()
        #replace all occurrences of the required string
        data = data.replace('yaml_path', ypath)
        
       
    with open(f"{proj_path}pepatac.sh", "w") as fh:
        fh.write(data)
        
  #  print(proj_path)
    

In [10]:
#write_pepatac_shell(DF, 80)

In [6]:
def write_pepatac_yaml(pep_DF, start_row):
    
    base_path = "/projects/ps-epigen/users/rlan/ATACSeq/PEPATAC/FFS/"
    dlead = DF[start_row:start_row + 1].squeeze()[0]
    pname = DF[start_row:start_row + 1].squeeze()[1]
    proj_path = base_path + dlead + "/" + pname + "/"
    stable = proj_path + "datasheet.csv"
    opath = proj_path + "out"
    ipath = DF[start_row:start_row + 1].squeeze()[6]
    
    
    # Replace strings in file with extracted idir & pdir
    with open("/projects/ps-epigen/users/rlan/ATACSeq/bot/project.temp.yaml", 'rt') as fh:
        data = fh.read()
        data = data.replace('Temp_Proj_Name', pname)
        data = data.replace('sample_table_path', stable)
        data = data.replace('output_path', opath)
        data = data.replace('input_path', ipath)
        
    with open(f"{proj_path}project.yaml", "w") as fh:
        fh.write(data)
        
        print(proj_path)
        

In [12]:
#write_pepatac_yaml(DF, 80)

In [ ]:
# For yaml we need 
# 1. Project Name
# 2. sample table path
# 3. output path
# 4 R1_Path & R2_Path

In [ ]:
### Functions to create. 
# 1. Format project.yaml
# 2. format pepatac.sh
# 3. Run pepatac
# 4. Find pepatac projects 


In [7]:
def find_pep_projects(PDF):
    ## Step 2. Identify rows of the sheet that are not completed
    no_rows = PDF[PDF['Completed'] =='No']
    
    # 3. Among the not completed rows, identify start and stop rows for each project present
    # 4. Create 2d list of each start, stop
    projects = np.unique(no_rows['Project Name'])
    proj_start_stops = list()

    for p in projects:
        tpdf = no_rows[no_rows['Project Name'] == p]
        proj_index = tpdf.index
        # + 1 because python shenanigans 
        start_stop = [proj_index[0], proj_index[-1] + 1]
        proj_start_stops.append(start_stop)
    
    return(proj_start_stops, projects)


In [10]:
indys, projects = find_pep_projects(DF)
indys, projects

([[339, 343]], array(['Julia_QY_2134-2137_2'], dtype=object))

In [11]:
##### Edit #####
def run_pepatac(pep_DF, start_row):
    # Format project path
    base_path = "/projects/ps-epigen/users/rlan/ATACSeq/PEPATAC/FFS/"
    dlead = DF[start_row:start_row + 1].squeeze()[0]
    pname = DF[start_row:start_row + 1].squeeze()[1]
    proj_path = base_path + dlead + "/" + pname + "/"
    
    # Format qsub
    qsub_cmd = ['qsub', f"{proj_path}pepatac.sh"]
   # print(qsub_cmd)
    subprocess.run(qsub_cmd)

In [49]:
#run_pepatac(pep_DF=DF, start_row=186)

In [14]:
for i in indys:
    create_pep_sheet(DF, start_row=i[0], stop_row=i[1])
    
    write_pepatac_shell(DF, start_row=i[0])
    
    write_pepatac_yaml(DF, i[0])
    
    run_pepatac(DF, i[0])
    
    

/projects/ps-epigen/users/rlan/ATACSeq/PEPATAC/FFS/Qian/Julia_QY_2134-2137_2/
33104093.tscc-mgr7.local


In [ ]:
######## NOTE #
# In final version make os.makedirs(proj_path, exist_ok = True) exist_ok = FALSE to avoid previous output 
# from getting overwritten by sheet editor or pipeline errors 